In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)


CUDA available: True


In [15]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.activation = activation

        self.fc_layers = nn.ModuleList()
        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.fc_layers.append(nn.Linear(prev_size, hidden_size))
            prev_size = hidden_size

        self.output_layer = nn.Linear(prev_size, output_size)

    def forward(self, x):
        x = x.view(-1, self.input_size)
        for fc_layer in self.fc_layers:
            x = self.activation(fc_layer(x))
        x = self.output_layer(x)
        return x

In [16]:
from sklearn.datasets import load_linnerud

linnerud = load_linnerud()
X = linnerud.data
y = linnerud.target


# Print the shape of the input and output arrays
print("Input shape:", X.shape)
print("Output shape:", y.shape)
# print(y[0])

Input shape: (20, 3)
Output shape: (20, 3)


In [17]:
from sklearn.model_selection import train_test_split
import numpy as np
import time

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU')
else:
    device = torch.device('cpu')
    print('Using CPU')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into PyTorch tensors and move them to the GPU
X_train = torch.FloatTensor(X_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_train = torch.FloatTensor(y_train).to(device)
y_test = torch.FloatTensor(y_test).to(device)

# Define the model architecture and move it to the GPU
input_size = X_train.shape[1]
hidden_sizes = [128, 64, 128, 128]
try:
    output_size = y_train.shape[1]
except IndexError:
    output_size = 1
model = MLP(input_size, hidden_sizes, output_size, F.relu).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.MSELoss()

# Number of epochs
n_epochs = 1000

# Placeholder for losses
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

# Get the current time before the for loop
start_time = time.time()

for it in range(n_epochs):
    # zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    loss.backward()
    optimizer.step()

    # Save losses
    train_losses[it] = loss.item()

    # Test loss
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    test_losses[it] = test_loss.item()

    if (it + 1) % 50 == 0:
        print(f'Epoch {it+1}/{n_epochs}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

# Get the current time after the for loop
end_time = time.time()

# Calculate the time difference
time_difference = end_time - start_time

# Print the time difference
print("Time difference:", time_difference)


Using GPU


Epoch 50/1000, Train Loss: 2907.9958, Test Loss: 1207.4757
Epoch 100/1000, Train Loss: 2783.5312, Test Loss: 1359.6780
Epoch 150/1000, Train Loss: 2755.6401, Test Loss: 1362.7412
Epoch 200/1000, Train Loss: 2718.7529, Test Loss: 1364.8265
Epoch 250/1000, Train Loss: 2666.4492, Test Loss: 1369.1882
Epoch 300/1000, Train Loss: 2577.2070, Test Loss: 1367.3977
Epoch 350/1000, Train Loss: 2367.6140, Test Loss: 1355.8112
Epoch 400/1000, Train Loss: 1452.0508, Test Loss: 1307.1379
Epoch 450/1000, Train Loss: 701.7020, Test Loss: 590.6973
Epoch 500/1000, Train Loss: 215.7643, Test Loss: 609.0274
Epoch 550/1000, Train Loss: 192.7612, Test Loss: 421.3122
Epoch 600/1000, Train Loss: 176.8075, Test Loss: 387.9638
Epoch 650/1000, Train Loss: 164.3175, Test Loss: 343.0928
Epoch 700/1000, Train Loss: 153.4014, Test Loss: 295.8835
Epoch 750/1000, Train Loss: 154.1211, Test Loss: 332.7675
Epoch 800/1000, Train Loss: 131.1653, Test Loss: 224.8227
Epoch 850/1000, Train Loss: 121.2389, Test Loss: 216.7153